In [12]:
# Tiago Moreira Trocoli da Cunha
# 226078
from __future__ import division
import numpy as np
from sklearn.svm import SVR

# matrix of 404 sample with 13 variables.
Xtreino = np.load("Xtreino5.npy")
Ytreino = np.load("ytreino5.npy")

# matrix of 102 tests with 13 variables.
Xteste = np.load("Xteste5.npy")
Yteste = np.load("yteste5.npy")    

# mean squared error
def mean_squared_error(X,Y):
    m = np.size(X)
    return (1.0/m)*(np.sum((X-Y)**2))

In [66]:
# 1) random search

# vector of mean squared erros (MSE)
mse = np.zeros(120,dtype=np.float64)

# vector of parameters
params = np.zeros((120,3),dtype=np.float64)

# generate parameters uniformaly
c        = np.random.uniform(2**(-5), 2**(15),120)
gamma    = np.random.uniform(2**(-15),2**3, 120)
epsilon  = np.random.uniform(0.05,1.0, 120)

for i in range(120):
    
    clf = SVR(kernel='rbf', gamma=gamma[i], C=c[i], epsilon=epsilon[i])
    clf.fit(Xtreino, Ytreino)
    mse[i] = mean_squared_error(clf.predict(Xteste),Yteste)
    params[i] = [clf.get_params()['C'], clf.get_params()['epsilon'], clf.get_params()['gamma']]
    
min_idx = np.argmin(mse)
param   = params[min_idx]

print "Minimum Mean Squared Error: ", mse[min_idx]
print "C = ", param[0]
print 'epsilon = ', param[1]
print "gamma = ", param[2]

Minimum Mean Squared Error:  50.83517760360566
C =  30312.107118378884
epsilon =  0.18228006191494311
gamma =  0.06595548500545227


In [25]:
# 2) Grid seach
from sklearn.model_selection import GridSearchCV

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 
                     'gamma': np.linspace(2**(-15),2**3, 5),
                     'C': np.linspace(2**(-5), 2**(15),5),
                     'epsilon': np.linspace(0.05,1.0, 5)}]

clf = GridSearchCV(SVR(), tuned_parameters, cv=5, scoring='neg_mean_squared_error',iid='False')
clf.fit(Xtreino, Ytreino)
mse = mean_squared_error(clf.predict(Xteste),Yteste)
print "Minimum Mean Squared Error: ", mse
print clf.best_params_

Minimum Mean Squared Error:  13.86700621013392
{'epsilon': 1.0, 'C': 16384.015625, 'gamma': 3.0517578125e-05, 'kernel': 'rbf'}


In [26]:
# 3) Otimizacao bayesiana
import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK

# return the mean squared error
def function(args):
    gamma, C, epsilon = args
    clf    = SVR(kernel='rbf', gamma=gamma, C=C, epsilon=epsilon)
    clf.fit(Xtreino, Ytreino)
    return mean_squared_error(clf.predict(Xteste),Yteste)

# defining the search space, we'll explore this more later
space = [hp.uniform('gamma', 2**(-15),2**3), 
         hp.uniform('C', 2**(-5), 2**(15)), 
         hp.uniform('epsilon', 0.05,1.0)]

# calling the hyperopt function
best = fmin(function,space,algo=tpe.suggest,max_evals=125)

print best

100%|██████████| 125/125 [00:03<00:00, 32.88it/s, best loss: 30.0741277169]
{'epsilon': 0.6111793828581504, 'C': 4001.914584128272, 'gamma': 0.008261913272106671}


In [25]:
# 4) PSO

# Encontre as constantes que multiplicam o momento ou inercia, a direcao do melhor ponto 
# da particula e a direcao do melhor global.

# Resposta:
# phip = Scaling factor to search away from the particle’s best known position
# phig = Scaling factor to search away from the swarm’s best known position
# omega = Particle velocity scaling factor

from pyswarm import pso

# cost =  mean squared error
def cost(x):
    clf = SVR(kernel='rbf', gamma=x[0], C=x[1], epsilon=x[2])
    clf.fit(Xtreino, Ytreino)
    return mean_squared_error(clf.predict(Xteste),Yteste)

lb = [2**(-15), 2**(-5), 0.05]
ub = [2**3, 2**(15), 1.0]

xopt, fopt = pso(cost, lb, ub, maxiter=11, swarmsize=11)

print "Minimum Mean Squared Error: ", fopt
print "C = ", xopt[1]
print 'epsilon = ', xopt[2]
print "gamma = ", xopt[0]

Stopping search: maximum iterations reached --> 11
Minimum Mean Squared Error:  11.431394554060141
C =  18583.344465570797
epsilon =  0.05
gamma =  3.0517578125e-05


In [24]:
# 5) Simulated annealing
# Como nao entendi direito o simannel, usei o dual_annealing da biblioteca scipy.optimize.
# quando o no_local_search=True, o algoritmo irá funcionar como tradicional 
# Generalized Simulated Annealing.

# http://scipy.github.io/devdocs/generated/scipy.optimize.dual_annealing.html#scipy.optimize.dual_annealing

from scipy.optimize import dual_annealing

def objective(args):
    gamma, C, epsilon = args
    clf = SVR(kernel='rbf', gamma=gamma, C=C, epsilon=epsilon)
    clf.fit(Xtreino, Ytreino)
    return  mean_squared_error(clf.predict(Xteste),Yteste)

    
lw = [2**(-15), 2**(-5), 0.05]
up = [2**3, 2**(15), 1.0]

minima = 2147483648
args = np.array([0,0,0])

for i in range(0,125):
    result = dual_annealing(objective, bounds=list(zip(lw, up)), no_local_search=True, maxiter = 10)

    if(minima > result.fun):
        minima = result.fun
        args   = result.x
        
print "Minimum Mean Squared Error: ", minima
print "C = ", args[1]
print 'epsilon = ', args[2]
print "gamma = ", args[0]

Minimum Mean Squared Error:  29.851263895302164
C =  9786.807449296117
epsilon =  0.10436806903015779
gamma =  0.01287783682346344
